In [1]:
pip install tronpy


  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached regex-2025.7.34-cp313-cp313-win_amd64.whl.metadata (41 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp313-cp313-win_amd64.whl.metadata (6.9 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached anyio-4.10.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached charset_normalizer-3.4.3-cp313-cp313-win_amd64.whl.metadata (37 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.

In [ ]:
from tronpy import Tron
from tronpy.keys import PrivateKey

# Connect to Nile testnet
client = Tron(network="nile")

# Replace with your sender private key
private_key_hex = "430fb0f7f75a9a992777068de1c2812360baa222ceea79c5acad818068f4b985"
priv_key = PrivateKey(bytes.fromhex(private_key_hex))

# Get sender address from private key
sender_addr = priv_key.public_key.to_base58check_address()

def send_payment(sender_addr, receiver_addr, amount_trx, commission_wallet):
    """
    Send payment with 1% commission deducted.
    :param sender_addr: TRON address of sender
    :param receiver_addr: TRON address of merchant
    :param amount_trx: total amount in TRX
    :param commission_wallet: your gateway wallet address
    """
    commission = amount_trx * 0.01
    merchant_amount = amount_trx - commission

    # Convert TRX to SUN (1 TRX = 1,000,000 SUN)
    merchant_sun = int(merchant_amount * 1_000_000)
    commission_sun = int(commission * 1_000_000)

    # Transaction 1: Pay Merchant
    txn1 = (
        client.trx.transfer(sender_addr, receiver_addr, merchant_sun)
        .build()
        .sign(priv_key)
    )
    txn1_ret = txn1.broadcast().wait()

    # Transaction 2: Pay Commission
    txn2 = (
        client.trx.transfer(sender_addr, commission_wallet, commission_sun)
        .build()
        .sign(priv_key)
    )
    txn2_ret = txn2.broadcast().wait()

    return {
        "merchant_paid": merchant_amount,
        "commission": commission,
        "txn1": txn1_ret,
        "txn2": txn2_ret,
    }


# Example Usage
result = send_payment(
    sender_addr=sender_addr,
    receiver_addr="TV4f8GuDe5JkfryegWWuYGM6MQEoe75HxE",   # merchant TRON address
    amount_trx=1000,                        # sending TRX
    commission_wallet="TV4f8GuDe5JkfryegWWuYGM6MQEoe75HxE" # your gateway wallet
)
print(result)


{'merchant_paid': 990.0, 'commission': 10.0, 'txn1': {'id': '1d90af8ea7b3b14e73e5c1a8c7cc602126766aac1e29194f6df071294589bd96', 'blockNumber': 60018299, 'blockTimeStamp': 1756465350000, 'contractResult': [''], 'receipt': {'net_usage': 269}}, 'txn2': {'id': '36ad4e9639706a2045214364d3bae85d7c7edfa06618c455d59ce4eec3a5c3b3', 'fee': 268000, 'blockNumber': 60018301, 'blockTimeStamp': 1756465356000, 'contractResult': [''], 'receipt': {'net_fee': 268000}}}


In [12]:
from tronpy import Tron

# Connect to Nile Testnet
client = Tron(network="nile")

def get_balance(address: str):
    """
    Get the TRX balance of a TRON account.
    :param address: TRON wallet address (starts with T...)
    :return: balance in TRX
    """
    try:
        # fetch balance in SUN
        balance_sun = client.get_account_balance(address)
        
        # balance is already in TRX (tronpy auto converts SUN → TRX)
        return balance_sun
    
    except Exception as e:
        return f"Error: {e}"


# Example usage
if __name__ == "__main__":
    my_address = "TGmWEgLPkNTMvM67wbmFLqrR47CtwSHhsF"
    print(f"Balance of {my_address}: {get_balance(my_address)} TRX")


Balance of TGmWEgLPkNTMvM67wbmFLqrR47CtwSHhsF: 8089.37966 TRX


# generate QR

In [8]:
pip install qrcode[pil]


  Using cached pillow-11.3.0-cp313-cp313-win_amd64.whl.metadata (9.2 kB)
Using cached pillow-11.3.0-cp313-cp313-win_amd64.whl (7.0 MB)

   ---------------------------------------- 0/2 [qrcode]
   ---------------------------------------- 0/2 [qrcode]
   ---------------------------------------- 0/2 [qrcode]
   -------------------- ------------------- 1/2 [pillow]
   -------------------- ------------------- 1/2 [pillow]
   -------------------- ------------------- 1/2 [pillow]
   -------------------- ------------------- 1/2 [pillow]
   -------------------- ------------------- 1/2 [pillow]
   -------------------- ------------------- 1/2 [pillow]
   -------------------- ------------------- 1/2 [pillow]
   -------------------- ------------------- 1/2 [pillow]
   -------------------- ------------------- 1/2 [pillow]
   -------------------- ------------------- 1/2 [pillow]
   -------------------- ------------------- 1/2 [pillow]
   ---------------------------------------- 2/2 [pillow]

Note: yo

In [9]:
import qrcode

# Your text (can be TRON address, payment link, or any text)
data = "TGmWEgLPkNTMvM67wbmFLqrR47CtwSHhsF"  

# Generate QR code
qr = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_H,
    box_size=10,
    border=4,
)
qr.add_data(data)
qr.make(fit=True)

# Save QR code as image
img = qr.make_image(fill_color="black", back_color="white")
img.save("qrcode.png")

print("QR Code generated and saved as qrcode.png")


QR Code generated and saved as qrcode.png
